## Lies `heights-weights.csv` in einen DataFrame

## Visualisiere die Verteilung von `Height` getrennt nach `Sex`

## Data Cleaning

Entferne alle `NaN` Werte folgendermaßen:
* entferne alle Observations, bei denen das Geschlecht unbekannt ist
* ergänze numerische Features mit einem Mittelwert des jeweiligen Geschlechts

## Training

* verwende 20% Testdaten und `random_state=42`
* predicte `Height`. Dabei sind zwei verschiedene Vorgangsweisen zu implementieren und zu vergleichen:
    1. mit Hilfe einer einfachen linearen Regression ist `Height` **nur** aus `Weight` zu predicten
    2. neben `Weight` ist auch `Sex` zu verwenden.

## Evaluierung (nur das Modell mit beiden Features)

* Stelle die Verteilung der Fehler in einem `distplot` dar
* Predicte deine eigenen Daten